<a href="https://colab.research.google.com/github/KhotNoorin/Algorithm-practice/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RNN code for Text Generation

Name: Noorin Nasir Khot

ID:2024KPAD1007

In [ ]:
import numpy as np
import random

In [ ]:
# Step 1: Dataset
dataset = [
    "I love machine learning.",
    "Text generation is fascinating.",
    "Artificial intelligence is evolving.",
    "I am learning deep learning.",
    "Recurrent neural networks are great for sequential data.",
    "Natural language processing is a subfield of AI.",
    "Data science involves many techniques.",
    "The weather today is sunny and warm.",
    "Python is a great programming language.",
    "I am studying for my exams.",
    "The dog chased the ball.",
    "He ran fast down the street.",
    "I enjoy coding in Python.",
    "The city is very crowded in the morning.",
    "I prefer coffee over tea.",
    "She loves reading books about history.",
    "My favorite color is blue.",
    "They played basketball in the park.",
    "I want to visit Paris someday.",
    "The sunset was beautiful last evening.",
    "We are learning about convolutional neural networks.",
    "This tutorial is very helpful for beginners.",
    "The movie was an incredible experience.",
    "She studied hard for her final project.",
    "I am going to the gym this evening."
]

In [ ]:
# Step 2: Preprocessing the text
words = ' '.join(dataset).lower().split()  # Convert to lowercase and split into words

In [ ]:
# Create a vocabulary
vocab = sorted(set(words))
vocab_size = len(vocab)

In [ ]:
# Create word-to-index and index-to-word mappings
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for idx, word in enumerate(vocab)}

In [ ]:
# Convert dataset sentences to sequences of integers
sequences = [np.array([word_to_idx[word] for word in sentence.lower().split()]) for sentence in dataset]

In [ ]:
# Prepare X (input sequences) and y (next word predictions)
X_data = []
y_data = []

In [ ]:
for sequence in sequences:
    for i in range(1, len(sequence)):
        X_data.append(sequence[:i])  # Input is the first i words
        y_data.append(sequence[i])   # Output is the next word

In [ ]:
# Padding sequences to make them the same length
max_seq_len = max(len(seq) for seq in X_data)
X_data = [np.pad(seq, (max_seq_len - len(seq), 0), mode='constant') for seq in X_data]
X_data = np.array(X_data)
y_data = np.array(y_data)

In [ ]:
# Step 3: Define a simple RNN model
hidden_size = 10
learning_rate = 0.01
epochs = 1000

In [ ]:
# Initialize weights and biases
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01  # Input to hidden weights
Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # Hidden to hidden weights
Why = np.random.randn(vocab_size, hidden_size) * 0.01  # Hidden to output weights
bh = np.zeros((hidden_size, 1))  # Hidden bias
by = np.zeros((vocab_size, 1))  # Output bias

In [ ]:
# Helper function for forward pass
def forward_pass(X, h_prev):
    # Compute hidden state
    h = np.tanh(np.dot(Wxh, X) + np.dot(Whh, h_prev) + bh)
    # Compute output
    y = np.dot(Why, h) + by
    return y, h

In [ ]:
# Loss function (softmax cross-entropy)
def loss(y_pred, y_true):
    return -np.log(np.exp(y_pred[y_true]) / np.sum(np.exp(y_pred)))

In [ ]:
# Step 4: Train the RNN using gradient descent
for epoch in range(epochs):
    total_loss = 0
    h_prev = np.zeros((hidden_size, 1))  # Initial hidden state

    for i in range(len(X_data)):
        X = np.zeros((vocab_size, 1))  # One-hot encoded input
        X[X_data[i][0]] = 1

        target = y_data[i]

        # Forward pass
        y_pred, h = forward_pass(X, h_prev)

        # Calculate loss (cross-entropy)
        total_loss += loss(y_pred, target)

        # Backpropagation (Gradient descent)
        dL_dy = np.exp(y_pred) / np.sum(np.exp(y_pred))  # Softmax derivative
        dL_dy[target] -= 1

        # Gradients for Why, by
        dL_dWhy = np.dot(dL_dy, h.T)
        dL_dby = dL_dy

        # Gradients for hidden layer (h)
        dL_dh = np.dot(Why.T, dL_dy)
        dL_dhraw = (1 - h * h) * dL_dh  # Derivative of tanh

        # Gradients for Wxh, Whh, bh
        dL_dWxh = np.dot(dL_dhraw, X.T)
        dL_dWhh = np.dot(dL_dhraw, h_prev.T)
        dL_dbh = dL_dhraw

        # Update weights and biases using gradient descent
        Wxh -= learning_rate * dL_dWxh
        Whh -= learning_rate * dL_dWhh
        Why -= learning_rate * dL_dWhy
        bh -= learning_rate * dL_dbh
        by -= learning_rate * dL_dby

        # Set h_prev for the next iteration
        h_prev = h

    # Print loss every 100 epochs
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {total_loss}')

Epoch 0, Loss: [584.2232238]
Epoch 100, Loss: [555.83788938]
Epoch 200, Loss: [546.28716184]
Epoch 300, Loss: [492.60866035]
Epoch 400, Loss: [374.4695367]
Epoch 500, Loss: [325.35538455]
Epoch 600, Loss: [297.55315642]
Epoch 700, Loss: [334.88081792]
Epoch 800, Loss: [337.53965615]
Epoch 900, Loss: [300.99287194]


In [ ]:
# Step 5: Generate text using the trained RNN model
def generate_text(seed_word, length=20):
    # Convert seed_word to lowercase to match the preprocessed data
    seed_word = seed_word.lower()

    # Check if seed_word is in the vocabulary
    if seed_word not in word_to_idx:
        raise ValueError(f"Seed word '{seed_word}' not found in the vocabulary.")

    word_idx = word_to_idx[seed_word]
    h_prev = np.zeros((hidden_size, 1))
    generated_text = [seed_word]

    for _ in range(length):
        X = np.zeros((vocab_size, 1))
        X[word_idx] = 1
        y_pred, h = forward_pass(X, h_prev)

        # Sample the next word
        prob = np.exp(y_pred) / np.sum(np.exp(y_pred))  # Softmax
        word_idx = np.random.choice(range(vocab_size), p=prob.ravel())

        generated_text.append(idx_to_word[word_idx])
        h_prev = h

    return ' '.join(generated_text)

In [ ]:
# Test text generation
seed_word = "Natural"  # works with the lowercase "i"
generated_sentence = generate_text(seed_word, length=20)
print("Generated Text: ", generated_sentence)

Generated Text:  natural morning. language processing many techniques. prefer language science involves techniques. morning. weather language processing tea. data. warm. language science involves


In [ ]:
seed_word = "I"
generated_sentence = generate_text(seed_word, length=20)
print("Generated Text: ", generated_sentence)

Generated Text:  i morning. language coffee ai. techniques. exams. language science processing techniques. language. weather processing involves many data. prefer prefer science involves
